# Group Details

## Group Name:

### Student 1:

### Student 2:

### Student 3:

# Loading Data and Preliminaries

In [1]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import torch

In [2]:
def load_array(filename, task):
    datapoint = np.load(filename)
    if task == 'task 1':
        initial_state = datapoint['initial_state']
        terminal_state = datapoint['terminal_state']
        return initial_state, terminal_state
    elif task == 'task 2' or task == 'task 3':
        whole_trajectory = datapoint['trajectory']
        # change shape: (num_bodies, attributes, time) ->  num_bodies, time, attributes
        whole_trajectory = np.swapaxes(whole_trajectory, 1, 2)
        initial_state = whole_trajectory[:, 0]
        target = whole_trajectory[:, 1:, 1:]  # drop the first timepoint (second dim) and mass (last dim) for the prediction task
        return initial_state, target
    else:
        raise NotImplementedError("'task' argument should be 'task 1', 'task 2' or 'task 3'!")


In [3]:
import os
"""
This cell gives an example of loading a datapoint with numpy for task 1.

The arrays returned by the function are structures as follows:
initial_state: shape (n_bodies, [mass, x, y, v_x, v_y])
terminal_state: shape (n_bodies, [x, y])

"""
example = load_array('data/task 1/train/trajectory_0.npz', task='task 1')

initial_state, terminal_state = example
print(f'shape of initial state (model input): {initial_state.shape}')
print(f'shape of terminal state (to be predicted by model): {terminal_state.shape}')

body_idx = 4
print(f'The initial x-coordinate of the body with index {body_idx} in this trajectory was {initial_state[body_idx, 1]}')

shape of initial state (model input): (8, 5)
shape of terminal state (to be predicted by model): (8, 2)
The initial x-coordinate of the body with index 4 in this trajectory was -2.7586106638591836


In [4]:
def get_data(folder):
    file_list = os.listdir(folder)

    inputs = []
    targets = []

    for file_name in file_list:
        file_path = os.path.join(folder, file_name)
        input, target = load_array(file_path, 'task 1')
        inputs.append(torch.from_numpy(input))
        targets.append(torch.from_numpy(target))

    return inputs, targets

In [5]:
X_train , y_train = get_data('D:/Master/Y1/Deep learning/Deep-learning/assignment_2/data/task 1/train')
X_test, y_test = get_data('D:/Master/Y1/Deep learning/Deep-learning/assignment_2/data/task 1/test')

In [6]:
from torch_geometric.data import Data

def make_graph(data):
       result = []
       for i in range(len(data)):
              x = data[i]
              num_bodies = x.shape[0]
              # create a tensor for the edges between nodes
              edge_index = torch.tensor([[i, j] for i in range(num_bodies) for j in range(num_bodies) if i != j], dtype=torch.long).t().contiguous()

              # create a PyG Data instance for the graph
              graph = Data(x=x, edge_index=edge_index)
              result.append(graph)
       return result

In [7]:
X_test_graph = make_graph(X_test)
X_train_graph = make_graph(X_train)
y_train_graph = make_graph(y_train)
y_test_graph = make_graph(y_test)

In [8]:
y_train_diff = [X_train[i][:, 1:3] - y_train[i] for i in range(len(X_train))]
y_test_diff = [X_test[i][:, 1:3] - y_test[i] for i in range(len(X_test))]

In [9]:
def get_edge_attr(data):
    num_bodies = data.shape[0]

    # We create a complete graph, so each body is connected to every other body
    edge_index = torch.tensor([[i, j] for i in range(num_bodies) for j in range(num_bodies) if i != j]).T

    # Calculate the distances and forces for each pair of bodies
    differences = data[edge_index[1]] - data[edge_index[0]]
    differences = differences[:,1:3]

    return differences

In [10]:
# def get_edge_attr_y(data):
#     num_bodies = data.shape[0]
#     edge_index = torch.tensor([[i, j] for i in range(num_bodies) for j in range(num_bodies) if i != j]).T
#     differences = data[edge_index[1]] - data[edge_index[0]]
#     differences = differences[:,0:2]
#     return differences


In [11]:
X_train_diff =  [torch.cat((X_train[i][:,:1], X_train[i][:,3:]), dim = 1) for i in range(len(X_train))]
X_train_diff_graph = make_graph(X_train_diff)
X_test_diff =  [torch.cat((X_test[i][:,:1], X_test[i][:,3:]), dim = 1) for i in range(len(X_test))]
X_test_diff_graph = make_graph(X_test_diff)

In [12]:
data_train = [Data(x=graph.x.to(torch.float32), edge_index=graph.edge_index, edge_attr=get_edge_attr(x).to(torch.float32), y=y.to(torch.float32)) for graph, y, x in zip(X_train_diff_graph, y_train_diff,X_train)]
data_test = [Data(x=graph.x.to(torch.float32), edge_index=graph.edge_index, edge_attr=get_edge_attr(x).to(torch.float32), y=y.to(torch.float32)) for graph, y, x in zip(X_test_diff_graph, y_test_diff,X_test)]

In [13]:
from torch_geometric.data import DataLoader

train_loader = DataLoader(data_train,batch_size=16, shuffle=True)
test_loader = DataLoader(data_test, batch_size=16, shuffle=True)

c:\Users\20172048\AppData\Local\Programs\Python\Python310\lib\site-packages\torch_geometric\deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [14]:
import torch_geometric.data as pyg_data


In [15]:
import torch
import torch.nn as nn

class ReadoutLayer(nn.Module):
    def __init__(self, edge_feature_dim, output_dim):
        super(ReadoutLayer, self).__init__()
        self.decoder = nn.Linear(edge_feature_dim, output_dim)

    def forward(self, nodes): 
        # pos_diff = torch.stack([edge_attr[i:i+(nodes.shape[0]-1)].sum(dim=0) for i in range(0, edge_attr.shape[0],nodes.shape[0]-1)]).to(torch.float32)
        return nodes[:,1:3]

class EdgeUpdateLayer(nn.Module):
    def __init__(self, edge_attr_dim):
        super(EdgeUpdateLayer, self).__init__()
        # self.update_forces = nn.Sequential(
        #     nn.Linear(5, 256),
        #     nn.Linear(256,1024),
        #     nn.Linear(1024,256),
        #     nn.Linear(256, 2)
        # )


    def forward(self, nodes, edge_attr):
        # print(nodes[:, :3])
        # print(nodes.shape[0])
        return get_edge_attr(nodes)
        # nodes_exp = nodes.repeat(nodes.shape[0]-1,1)
        # # print(nodes_exp.shape[0])
    
        # return self.update_forces(torch.cat([edge_attr,nodes_exp[:,:3]], dim = 1))

class NodeUpdateLayer(nn.Module):
    def __init__(self, node_feature_dim, edge_feature_dim):
        super(NodeUpdateLayer, self).__init__()
        self.update_velocities = nn.Sequential(
            nn.Linear(node_feature_dim + edge_feature_dim, 64), # input includes forces
            nn.ReLU(),
            nn.Linear(64, 2) # output dimension is 2 for (velx, vely)
        )
        self.update_forces = nn.Sequential(
            nn.Linear(edge_feature_dim,64),
            nn.ReLU(),
            nn.Linear(64, 2)
        )
        # self.update_positions = nn.Sequential(
        #     nn.Linear(node_feature_dim + 2, 256), # input includes updated velocities
        #     nn.Linear(256, 2) # output dimension is 2 for (posx, posy)

        # )

    def forward(self, nodes, edge_attr):
        # print(len(edge_attr))
        # updated_forces = torch.stack([edge_attr[i:i+(nodes.shape[0]-1)].sum(dim=0) for i in range(0, edge_attr.shape[0],nodes.shape[0]-1)]).to(torch.float32)
        updated_forces = self.update_forces(edge_attr).to(torch.float32)
        updated_forces = torch.stack([updated_forces[i:i+(nodes.shape[0]-1)].sum(dim=0) for i in range(0, updated_forces.shape[0],nodes.shape[0]-1)]).to(torch.float32)
        # print(nodes)
        # print(edge_attr)
        # print(updated_forces)
        # updated_forces = edge_attr
        updated_velocities = self.update_velocities(torch.cat([nodes, updated_forces], dim=1)).to(torch.float32)
        # updated_positions = self.update_positions(torch.cat([nodes, updated_velocities], dim=1)).to(torch.float32)
        return torch.cat([nodes[:, :1], updated_velocities], dim=1).to(torch.float32) # concatenate mass, and velocities



class GNN(nn.Module):
    def __init__(self, node_feature_dim, edge_feature_dim):
        super(GNN, self).__init__()
        self.edge_update = EdgeUpdateLayer(edge_feature_dim)
        self.node_update = NodeUpdateLayer(node_feature_dim, edge_feature_dim)
        self.readout = ReadoutLayer(edge_feature_dim, output_dim=2)

    def forward(self, batch):
        out = []

        # Iterate over all graphs in the batch
        for graph in pyg_data.Batch.to_data_list(batch):
            nodes = graph.x
            edges = graph.edge_index
            edge_attr = graph.edge_attr

            for T in range(5):  # T is the number of message passing steps
                nodes = self.node_update(nodes, edge_attr)
                edge_attr = self.edge_update(nodes, edge_attr).to(torch.float32)

            out.append(self.readout(nodes))

        return torch.cat(out)

In [16]:
# Loss function
criterion = nn.MSELoss()

model_task1 = GNN(3,2)

# Optimizer
optimizer = torch.optim.Adam(model_task1.parameters())

# Number of epochs
epochs = 100


for epoch in range(epochs):
    model_task1.train()

    total_loss = 0
    for batch in train_loader:
        targets_batch = batch.y

        optimizer.zero_grad()
        outputs = model_task1(batch)
        loss = criterion(outputs, targets_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    total_loss /= len(train_loader)

    model_task1.eval()

    total_test_loss = 0
    with torch.no_grad():
        for batch in test_loader:
            targets_batch = batch.y
            outputs = model_task1(batch)
            loss_test = criterion(outputs, targets_batch)
            total_test_loss += loss_test.item()

    total_test_loss /= len(test_loader)

    if (epoch+1) % 1 == 0:
        print ('Epoch [{}/{}], Train loss: {:.4f}, Test loss: {:.4f}'.format(epoch+1, epochs, total_loss, total_test_loss))

Epoch [1/100], Train loss: 12.2017, Test loss: 9.1325
Epoch [2/100], Train loss: 11.1122, Test loss: 8.6031
Epoch [3/100], Train loss: 10.8818, Test loss: 8.7129
Epoch [4/100], Train loss: 11.0104, Test loss: 8.6385
Epoch [5/100], Train loss: 10.8339, Test loss: 8.8273
Epoch [6/100], Train loss: 10.7307, Test loss: 8.7862
Epoch [7/100], Train loss: 10.7113, Test loss: 8.6683
Epoch [8/100], Train loss: 10.7988, Test loss: 8.4639
Epoch [9/100], Train loss: 10.5073, Test loss: 8.6841
Epoch [10/100], Train loss: 10.5700, Test loss: 9.1067
Epoch [11/100], Train loss: 10.5003, Test loss: 8.5708
Epoch [12/100], Train loss: 10.4137, Test loss: 8.8008
Epoch [13/100], Train loss: 10.6063, Test loss: 8.6699
Epoch [14/100], Train loss: 10.3604, Test loss: 8.5433
Epoch [15/100], Train loss: 10.4583, Test loss: 9.0027
Epoch [16/100], Train loss: 10.4310, Test loss: 8.9709
Epoch [17/100], Train loss: 10.4360, Test loss: 8.4797
Epoch [18/100], Train loss: 10.3727, Test loss: 8.5135
Epoch [19/100], Tra

In [17]:
"""
This cell gives an example of loading a datapoint with numpy for task 2 / 3.

The arrays returned by the function are structures as follows:
initial_state: shape (n_bodies, [mass, x, y, v_x, v_y])
remaining_trajectory: shape (n_bodies, time, [x, y, v_x, v_y])

Note that for this task, you are asked to evaluate performance only with regard to the predictions of the positions (x and y).
If you use the velocity of the remaining trajectory for training,
this use should be purely auxiliary for the goal of predicting the positions [x,y] over time. 
While testing performance of your model on the test set, you do not have access to v_x and v_y of the remaining trajectory.

"""

example = load_array('data/task 2_3/train/trajectory_0.npz', task='task 2')

initial_state, remaining_trajectory = example
print(f'shape of initial state (model input): {initial_state.shape}')
print(f'shape of terminal state (to be predicted by model): {remaining_trajectory.shape}')

body_idx = 2
time_idx = 30
print(f'The y-coordinate of the body with index {body_idx} at time with index {time_idx} in remaining_trajectory was {remaining_trajectory[body_idx, time_idx, 1]}')

test_example = load_array('data/task 2_3/test/trajectory_900.npz', task='task 3')
test_initial_state, test_remaining_trajectory = test_example
print(f'the shape of the input of a test data example is {test_initial_state.shape}')
print(f'the shape of the target of a test data example is {test_remaining_trajectory.shape}')
print(f'values of the test data example at time {time_idx}:\n {test_remaining_trajectory[:, time_idx]}')
print('note: velocity values are unobserved (NaNs) in the test data!')

shape of initial state (model input): (8, 5)
shape of terminal state (to be predicted by model): (8, 49, 4)
The y-coordinate of the body with index 2 at time with index 30 in remaining_trajectory was -0.3861544940435097
the shape of the input of a test data example is (8, 5)
the shape of the target of a test data example is (8, 49, 4)
values of the test data example at time 30:
 [[-1.11611543  3.21149953         nan         nan]
 [-0.2865083   4.30801877         nan         nan]
 [ 1.07701594 -8.12529269         nan         nan]
 [-0.92053478  3.13709551         nan         nan]
 [-3.96308297 -4.27733589         nan         nan]
 [ 2.33945401 -8.67733599         nan         nan]
 [-4.83949085  3.67854952         nan         nan]
 [ 0.31080159 -9.74720071         nan         nan]]
note: velocity values are unobserved (NaNs) in the test data!


# Data Handling and Preprocessing

In [18]:
#todo

# Model Implementation

In [19]:
#todo

# Model Training

In [20]:
#todo

# Evaluation

In [21]:
#todo